<a href="https://colab.research.google.com/github/atharvaballa/CBTCIP/blob/main/Creating_Payment_Receipts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import random
import datetime
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from google.colab import drive

def show_available_items():
    """Displays the list of available stationery items."""
    stationery_items = {
        1: "Pen", 2: "Pencil", 3: "Notebook", 4: "Eraser",
        5: "Ruler", 6: "Sharpener", 7: "Marker", 8: "Glue",
        9: "Stapler", 10: "Paper Clips"
    }
    fixed_prices = {
        "Pen": 20, "Pencil": 10, "Notebook": 75, "Eraser": 15,
        "Ruler": 30, "Sharpener": 20, "Marker": 40, "Glue": 45,
        "Stapler": 120, "Paper Clips": 7
    }
    print("Available Items:")

    # Adjust to right-align the entire "(Price: Rs. ...)" part
    for idx, item in stationery_items.items():
        price_str = f" (Price: Rs. {fixed_prices[item]:>3})"
        print(f"{idx:>2}. {item:<15}{price_str:>15}")  # Align the item name left and the price right

    return stationery_items, fixed_prices

def take_user_input(stationery_items, fixed_prices):
    """Takes user input for item selection and quantity."""
    user_items = []
    while True:
        try:
            item_no = int(input("Enter the item number you want to purchase (0 to finish): "))
            if item_no == 0:
                break
            if item_no not in stationery_items:
                print("Invalid item number. Please select from the available items.")
                continue
            item_name = stationery_items[item_no]
            quantity = int(input(f"Enter the quantity for {item_name}: "))
            if quantity <= 0:
                print("Quantity must be greater than 0. Try again.")
                continue
            price = fixed_prices[item_name]
            user_items.append({"name": item_name, "quantity": quantity, "price": price})
        except ValueError:
            print("Invalid input. Please enter numbers only.")
    return user_items

def create_receipt(data):
    """
    Creates a payment receipt as a PDF file and saves it to Google Drive.

    Args:
        data (dict): A dictionary containing receipt data.
    """
    #drive.mount('/content/drive')
    os.makedirs('/content/drive/MyDrive/CipherByte Tasks/Receipts', exist_ok=True)

    filename = '/content/drive/MyDrive/CipherByte Tasks/Receipts/receipt_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.pdf'
    doc = canvas.Canvas(filename, pagesize=letter)

    page_width, page_height = letter
    border_x, border_y = 50, 50
    border_width, border_height = page_width - 100, page_height - 100

    # Draw the border box
    p = doc.beginPath()
    p.moveTo(border_x, border_y)
    p.lineTo(border_x + border_width, border_y)
    p.lineTo(border_x + border_width, border_y + border_height)
    p.lineTo(border_x, border_y + border_height)
    p.close()
    doc.rect(border_x, border_y, border_width, border_height)

    # Receipt header
    doc.setFont("Helvetica-Bold", 16)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 30, "Payment Receipt")

    # Store information
    doc.setFont("Helvetica", 12)
    doc.drawString(border_x + 20, border_y + border_height - 60, f"Store Name: {data['store_name']}")
    doc.drawString(border_x + 20, border_y + border_height - 80, f"Store Address: {data['store_address']}")

    # Transaction details
    doc.setFont("Helvetica-Bold", 14)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 120, "Transaction Details")
    doc.setFont("Courier", 12)
    doc.drawString(border_x + 20, border_y + border_height - 150, f"Transaction ID: {data['transaction_id']}")
    doc.drawString(border_x + 20, border_y + border_height - 170, f"Date: {data['date']}")
    doc.drawString(border_x + 20, border_y + border_height - 190, f"Total Amount: Rs. {data['total_amount']:.2f}")

    # Itemized list
    doc.setFont("Helvetica-Bold", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + border_height - 220, "Items Purchased")
    doc.setFont("Courier", 10)

    # Right-aligning prices
    right_align_x = border_x + border_width - 100  # X position for right alignment
    y = border_y + border_height - 250
    for item in data['items']:
        subtotal = item['quantity'] * item['price']
        item_str = f"{item['quantity']} x {item['name']} @ Rs. {item['price']} each"
        doc.drawString(border_x + 20, y, item_str)
        doc.drawRightString(right_align_x, y, f"Rs. {subtotal:.2f}")
        y -= 20

    # Payment method
    doc.setFont("Helvetica-Bold", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + 100, "Payment Method")
    doc.setFont("Helvetica", 12)
    doc.drawCentredString(border_x + border_width / 2, border_y + 80, f"{data['payment_method']}")

    # Footer
    doc.setFont("Helvetica", 8)
    doc.drawCentredString(border_x + border_width / 2, border_y + 20, "This receipt is for your records only.")
    doc.save()
    print(f"Receipt saved successfully to {filename}")

if __name__ == "__main__":
    stationery_items, fixed_prices = show_available_items()
    user_items = take_user_input(stationery_items, fixed_prices)

    if not user_items:
        print("No items selected. Exiting.")
    else:
        total_amount = sum(item['quantity'] * item['price'] for item in user_items)
        receipt_data = {
            "store_name": "Stationery",
            "store_address": "India",
            "transaction_id": str(random.randint(10000, 99999)),
            "date": datetime.datetime.now().strftime('%Y-%m-%d'),
            "total_amount": total_amount,
            "items": user_items,
            "payment_method": "Cash"
        }
        create_receipt(receipt_data)


Available Items:
 1. Pen             (Price: Rs.  20)
 2. Pencil          (Price: Rs.  10)
 3. Notebook        (Price: Rs.  75)
 4. Eraser          (Price: Rs.  15)
 5. Ruler           (Price: Rs.  30)
 6. Sharpener       (Price: Rs.  20)
 7. Marker          (Price: Rs.  40)
 8. Glue            (Price: Rs.  45)
 9. Stapler         (Price: Rs. 120)
10. Paper Clips     (Price: Rs.   7)
Enter the item number you want to purchase (0 to finish): 1
Enter the quantity for Pen: 4
Enter the item number you want to purchase (0 to finish): 2
Enter the quantity for Pencil: 3
Enter the item number you want to purchase (0 to finish): 4
Enter the quantity for Eraser: 5
Enter the item number you want to purchase (0 to finish): 
Invalid input. Please enter numbers only.
Enter the item number you want to purchase (0 to finish): 0
Receipt saved successfully to /content/drive/MyDrive/CipherByte Tasks/Receipts/receipt_20241127_113012.pdf
